In [80]:
from __future__ import division
from pandas import *
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import scipy


plt.style.use('ggplot')

%matplotlib inline
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 64.3 ms


In [81]:
data_path = '../boots_dataset/'

time: 20 ms


### load features -- drop all the duplicate images

In [82]:
features = read_csv(data_path + 'boots_features.csv', 
                    index_col = 'asin')\
                .drop_duplicates()
features.shape

(7307, 4096)

time: 11.6 s


#### save those back

In [92]:
features.to_csv(data_path + 'boots_features_dedup.csv')

time: 36.8 s


In [84]:
features.head()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
asin,,,,,,,,,,,,,,,,,,,,,
B009XSN76O,0.0000,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,1.9695,0.0000,...,0.0,0.0,0.0,0.0,0.0000,1.2840,0.0,1.0314,0.0000,0.0
B0092OQN4C,1.2842,0.0000,0.0000,0.0,0.000,0.0,0.8610,1.8311,0.0000,1.7899,...,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.5421,0.6577,0.0
B00AJ1A23A,0.0000,3.2907,1.7814,0.0,0.000,0.0,0.4033,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.5277,0.0000,0.0,0.0000,0.0000,0.0
B005IYS5ZA,0.0000,0.8051,0.0000,0.0,0.000,0.1,0.0000,0.3484,0.0000,1.3334,...,0.0,0.0,0.0,0.0,0.0000,3.2731,0.0,0.0507,0.0000,0.0
B007KHXTXQ,0.0000,0.0000,2.9935,0.0,1.199,0.0,0.0000,0.0000,2.8597,0.0000,...,0.0,0.0,0.0,0.0,2.6146,0.0000,0.0,0.0201,0.0000,0.0


time: 58.6 ms


### load metadata

In [85]:
metadata = read_csv(data_path + 'boots_aws.csv', index_col = 'Unnamed: 0')
metadata = metadata[metadata.pic_downloaded == True]

time: 153 ms


In [86]:
metadata.head()

,title,brand,price,asin,related,imUrl,salesRank,categories,description,pic_downloaded,boots,sex,image_path
1139,Justin Boots Black Iguana Lizard,NaN,329.95,B0000A1R6F,NaN,http://ecx.images-amazon.com/images/I/51AutsCt...,{'Shoes': 249279},NaN,NaN,True,True,unknown,pics_boots/B0000A1R6F.jpg
1141,"""Justin Boots Men's Classic Western Boot""",NaN,NaN,B0000A1RDO,"{'also_bought': ['B000E9626I','B0041T42DC']'al...",http://ecx.images-amazon.com/images/I/41zrtA71...,{'Shoes': 53027},NaN,NaN,True,True,men,pics_boots/B0000A1RDO.jpg
1142,"""Justin Boots Men's Classic Western Boot""",NaN,NaN,B0000A1REG,"{'also_bought': ['B000E9626I','B0041T42DC']'al...",http://ecx.images-amazon.com/images/I/41zrtA71...,{'Shoes': 53027},NaN,NaN,True,True,men,pics_boots/B0000A1REG.jpg
2134,"""Children's Ostrich-Print Cowboy Boots-peanut ...",NaN,NaN,B0000CH8HL,"{'also_bought': ['B00551RXJK','B002LMHSEG','B0...",http://ecx.images-amazon.com/images/I/21M7uJMJ...,{'Shoes': 89070},NaN,NaN,True,True,men,pics_boots/B0000CH8HL.jpg
2785,"""Men's Itasca Waterproof Swampwalker II Rubber...",NaN,NaN,B0000WYVSA,"{'also_viewed': ['B002OSXDSC','B002BH4GAU','B0...",http://ecx.images-amazon.com/images/I/41S6OxKc...,{'Shoes': 89433},NaN,NaN,True,True,men,pics_boots/B0000WYVSA.jpg


time: 53 ms


#### add a category of rain/snow boots

In [87]:
metadata.sex.value_counts()

women       6709
men         2147
unknown     1945
children     337
unisex       106
Name: sex, dtype: int64

time: 29.1 ms


In [88]:
def sex_of_boots(x):
    for rain_word in ['rain','snow']:
        if rain_word in x:
            return 'weather'        
    for woman_word in ['women', 'woman', 'ladies', 'lady', 'girl']:
        if woman_word in x:
            return 'women'
    for man_word in ['men', 'man', 'boy']:
        if man_word in x:
            return 'men'
    for child_word in ['kid', 'child', 'youth', 'baby', 'infant']:
        if child_word in x:
            return 'children'
    for unisex_word in ['unisex', 'adult']:
        if unisex_word in x:
            return 'unisex'
    else:
        return 'unknown'

metadata.loc[:, 'sex'] = metadata.loc[:, 'title']\
                .apply(lambda x: sex_of_boots(x.lower()))

time: 71.8 ms


In [89]:
metadata.sex.value_counts()

women       5877
men         2005
unknown     1747
weather     1201
children     309
unisex       105
Name: sex, dtype: int64

time: 37.7 ms


### make a men's/women's metadata set

In [90]:
metadata.to_csv(data_path + 'boots_aws_160403.csv')

men = metadata[metadata.sex == 'men']
women = metadata[metadata.sex == 'women']

men.to_csv(data_path + 'metadata_men.csv')
women.to_csv(data_path + 'metadata_women.csv')

time: 360 ms


### make a men's/women's feature set

In [93]:
features_women = features[features.index.isin(women.asin)]
features_men = features[features.index.isin(men.asin)]
features_women.to_csv(data_path + 'features_women.csv')
features_men.to_csv(data_path + 'features_men.csv')

time: 25.5 s
